# Applying SOTA Transformers and CNNs to time series

In [ ]:
!pip -qq install tsai

In [ ]:
from tsai.all import *
computer_setup()

In [ ]:
import numpy as np
import pandas as pd
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
beep()

## Testing on a benchmarked dataset
https://zenodo.org/record/3902718#.YUSyeWYzZfU

In [ ]:
get_Monash_regression_list()

In [ ]:
dsid = "NewsHeadlineSentiment"
# X_train, y_train, X_valid, y_valid = get_Monash_regression_data(dsid, on_disk=True, split_data=True)
X, y, splits = get_Monash_regression_data(dsid, on_disk=True, split_data=False, force_download=True, verbose=True)

In [ ]:
beep()

In [ ]:
tfms  = [None, None]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits)

In [ ]:
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=64, batch_tfms=TSStandardize(by_var=True))

In [ ]:
dls.vars

In [ ]:
dls.c

In [ ]:
dls.len

In [ ]:
model = TST(dls.vars, dls.c, dls.len, res_dropout=0.3, fc_dropout=0.9)

In [ ]:
learn = Learner(dls, model, loss_func=nn.MSELoss(),metrics=mae,  cbs=ShowGraphCallback2())
learn.fit_one_cycle(2, 1e-4) 

## Applying to Crypto Asset Prices

In [ ]:
df=pd.DataFrame()
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        tmp=pd.read_csv(os.path.join(dirname, filename))
        df=pd.concat([df,tmp])
    
df.Date=pd.to_datetime(df.Date)

In [ ]:
print(df.Symbol.unique())

In [ ]:
df.head(2)

In [ ]:
Marketcap=pd.pivot_table(df,values='Marketcap',columns='Symbol',index='Date').pct_change()
Volume   =pd.pivot_table(df,values='Volume'   ,columns='Symbol',index='Date').pct_change()

Close    =pd.pivot_table(df,values='Close'    ,columns='Symbol',index='Date').pct_change()
Open     =pd.pivot_table(df,values='Open'     ,columns='Symbol',index='Date').pct_change()

High     =pd.pivot_table(df,values='High'     ,columns='Symbol',index='Date').pct_change()
Low      =pd.pivot_table(df,values='Low'      ,columns='Symbol',index='Date').pct_change()

Merge

In [ ]:
# col=['AAVE', 'ADA', 'ATOM', 'BNB', 'BTC', 'CRO', 'DOGE', 'DOT', 'EOS', 'ETH',
#        'LINK', 'LTC', 'MIOTA', 'SOL', 'TRX', 'UNI', 'USDC', 'USDT', 'WBTC',
#        'XEM', 'XLM', 'XMR', 'XRP']
col=['BTC']
df=Close[col].copy()
# df=Close[col].merge(Volume[col],left_index=True,right_index=True,suffixes=('','_Vol'))
# df=Marketcap[col].merge(Volume[col],
#                    left_index=True,
#                    right_index=True,
#                    suffixes=('_MarCap','_Vol')).merge(Open[col],
#                                                       left_index=True,
#                                                       right_index=True).merge(High[col],
#                                                                               left_index=True,
#                                                                               right_index=True,
#                                                                               suffixes=('_Open','_High')).merge(Low[col],
#                                                                                                              left_index=True,
#                                                                                                              right_index=True).merge(Close[col],
#                                                                                                                                      left_index=True,
#                                                                                                                                      right_index=True,
#                                                                                                                                      suffixes=('_Low',''))
# df=Marketcap[['BTC']].merge(Volume[['BTC']],left_index=True,right_index=True,suffixes=('_MarCap','_Vol')).merge(Close,left_index=True,right_index=True)
# df=Marketcap.merge(Volume,left_index=True,right_index=True,suffixes=('_MarCap','_Vol')).merge(Ret,left_index=True,right_index=True)


In [ ]:
df.head()

In [ ]:
df=df.loc["2018-01-01":]

In [ ]:
df.shape

In [ ]:
(df.BTC>0).value_counts()

Testing whether a library works or not

In [ ]:
df.BTC.mean()

In [ ]:
# df['target']=(df['BTC']>0.03).shift(-1).map(str) #for classification TEST
# df['target']=(df['BTC']>0.0027).map(str) #for classification TEST
df['target']=df['BTC'].shift(-1) #for regression
df.head(4)

In [ ]:
# df.target.value_counts()/len(df)

In [ ]:
df=df.iloc[1:-1,:]
# df.target.value_counts()/len(df)

In [ ]:
# df.fillna(0,inplace=True)

In [ ]:
df.plot()

In [ ]:
des=df.describe()
des=des.T
des

# Coding Exercise 1

Come up with a simple and good baseline to measure our model against

In [ ]:
#use average returns to predict next day's returns
abs(df.target-0.002737).mean()

In [ ]:
#use today's return to predict next day's return
abs(df.target-df.BTC).mean()

In [ ]:
get_x = df.columns[:-1]      # Indicates which are the columns that contain the x data.
get_y = ['target']             # In multivariate time series, you must indicate which is/are the y columns

window_length = 3         # window_length is usually selected based on prior domain knowledge or by trial and error
stride = 1                # None for non-overlapping (stride = window_length) (default = 1). This depends on how often you want to predict once the model is trained
start = 0                 # use all data since the first time stamp (default = 0)
horizon = 0               # horizon = number of future datapoints to predict. 0 for last step in the selected window. > 0 for future steps. List for several steps.
                            
X, y = SlidingWindow(window_length, stride=stride, start=start, get_x=get_x,  get_y=get_y, horizon=horizon)(df)
splits = get_splits(y, valid_size=.2, random_state=23, shuffle=False) # shuffle false since we are using one long time series
# tfms  = [None, Categorize()] # #Categoriztion
tfms  = [None, None] # Regression
dsets = TSDatasets(X, y, tfms=tfms, splits=splits)
dsets

visual examintation looks right

In [ ]:
check_data(X,y)

In [ ]:
X[0],y[0]

In [ ]:
# X[1]

In [ ]:
bs=256
batch_tfms=[TSStandardize(by_var=True)] #TSStandardize(mean=0,std=1)
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=bs, batch_tfms=batch_tfms)
dls.show_batch()

In [ ]:
x= next(iter(dls.train))
# x[0].data,x[1].data,
x[0].data[:3],x[1].data[:3],x[0].data.mean(),x[0].data.std()

In [ ]:
dls.vars, dls.c, dls.len

In [ ]:
model

In [ ]:
model = TSTPlus(dls.vars, dls.c, dls.len,
                res_dropout=0.3, fc_dropout=0.9,
                n_layers=1,
#                 d_model=1024,
#                 d_k=512,
#                 d_v=512,
#                 d_ff=4096,
               )#res_dropout=0.3, fc_dropout=0.9,
learn = Learner(dls, model, loss_func=nn.MSELoss(),metrics=mae)
# learn = Learner(dls, model, metrics=accuracy)#loss_func=LabelSmoothingCrossEntropyFlat(, cbs=[ShowGraphCallback2(),EarlyStoppingCallback(monitor='accuracy', min_delta=0.001, patience=30)])
learn.lr_find()

In [ ]:
learn = Learner(dls, model,  loss_func=nn.MSELoss(),metrics=mae,cbs=ShowGraphCallback2())

# F=F1Score()
# R=Recall()
# learn = Learner(dls, model, metrics=[R,F,accuracy],cbs=ShowGraphCallback2())

lr=1e-3
n_epochs=100
learn.fit_one_cycle(n_epochs, lr_max=lr);beep()

In [ ]:
# learn.recorder.values
learn.plot_metrics()

In [ ]:
learn.show_results()

In [ ]:
# Labeled test data
test_ds = dls.dataset.add_test(X[-1:], y[-1:])# In this case I'll use X and y, but this would be your test data
test_dl = dls.new(test_ds)
# next(iter(test_dl))

_, test_targets, test_preds = learn.get_preds(dl=test_dl, with_decoded=True, save_preds=None, save_targs=None)
abs(test_preds-test_targets)

In [ ]:
# learn.show_probas()

In [ ]:
# interp = ClassificationInterpretation.from_learner(learn)
# interp.plot_confusion_matrix()

now let's try inception time

In [ ]:
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=bs, batch_tfms=TSStandardize())
model = InceptionTimePlus(dls.vars, dls.c, dls.len)
# learn = Learner(dls, model, metrics=accuracy)#, loss_func=LabelSmoothingCrossEntropyFlat()
learn = Learner(dls, model,  loss_func=nn.MSELoss(),metrics=mae)
learn.fit_one_cycle(n_epochs, lr_max=lr);beep()

In [ ]:
learn.plot_metrics()

In [ ]:
_, test_targets, test_preds = learn.get_preds(dl=test_dl, with_decoded=True, save_preds=None, save_targs=None)
abs(test_preds-test_targets)

# Coding Exercise 2 - Can we improve performance?
Tweak either of TSTPlus and InceptionTimePlus `model` to improve results. Possible ways include
1. Changing the context length, i.e. how long a sequence the model has 
1. Training for more epochs
1. Use better learning rate(`lr`)
1. Have a deeper model
1. Add/Engineer Features

In [ ]:
#code

# Coding Exercise 3 - Can we improve performance by shuffling dataset?
Not a good idea.

In [ ]:
#code here

# Pre-Training InceptionTime and TST
Autoregressive denoising objective that does not require labels

In [ ]:
dls100  = get_ts_dls(X, y, splits=splits, tfms=tfms, batch_tfms=batch_tfms)
udls100 = get_ts_dls(X, splits=splits, tfms=tfms, batch_tfms=batch_tfms) # used in pretraining

In [ ]:
learn = ts_learner(udls100, InceptionTimePlus, cbs=[ShowGraph(), MVP(target_dir='./data/MVP', fname=f'Inception')])
learn.fit_one_cycle(50, 1e-2)

In [ ]:
learn.MVP.show_preds(sharey=True)

In [ ]:
learn = ts_learner(dls100, InceptionTimePlus, pretrained=True, weights_path=f'data/MVP/Inception.pth', metrics=nn.L1Loss())
for p in learn.model.parameters():
    p.requires_grad=False
print(f'{"trainable params once manually frozen":40}: {count_parameters(learn.model):8}')
learn.freeze()
print(f'{"trainable params after learn.freeze()":40}: {count_parameters(learn.model):8}')
learn.unfreeze()
print(f'{"trainable params learn.unfreeze()":40}: {count_parameters(learn.model):8}')

# self-supervised: fine-tuning with 100% labels

In [ ]:
n_epochs = 50
freeze_epochs = 10
n_tests = 2#10
_result = []
for i in range(n_tests):
    clear_output()
    if i > 0: print(f'{i}/{n_tests} MAE: {np.mean(_result):.3f} +/- {np.std(_result):.3f}')
    else: print(f'{i}/{n_tests}')
    learn = ts_learner(dls100, InceptionTimePlus, pretrained=True, weights_path=f'data/MVP/Inception.pth', metrics=nn.L1Loss())
    learn.fine_tune(n_epochs, base_lr=2e-2, freeze_epochs=freeze_epochs)
    _result.append(learn.recorder.values[-1][-1])
learn.plot_metrics()
print(f'\nMAE: {np.mean(_result):.3f} +/- {np.std(_result):.3f} in {n_tests} tests')

try for `TSTPlus`

In [ ]:
learn = ts_learner(udls100, TSTPlus, cbs=[ShowGraph(), MVP(target_dir='./data/MVP', fname=f'TSTPlus')])
learn.fit_one_cycle(50, 1e-2)

In [ ]:
learn.MVP.show_preds(sharey=True)

In [ ]:
learn = ts_learner(dls100, InceptionTimePlus, pretrained=True, weights_path=f'data/MVP/Inception.pth', metrics=mae)
for p in learn.model.parameters():
    p.requires_grad=False
print(f'{"trainable params once manually frozen":40}: {count_parameters(learn.model):8}')
learn.freeze()
print(f'{"trainable params after learn.freeze()":40}: {count_parameters(learn.model):8}')
learn.unfreeze()
print(f'{"trainable params learn.unfreeze()":40}: {count_parameters(learn.model):8}')

In [ ]:
# self-supervised: fine-tuning with 100% labels
n_epochs = 50
freeze_epochs = 10
n_tests = 2#10
_result = []
for i in range(n_tests):
    clear_output()
    if i > 0: print(f'{i}/{n_tests} MAE: {np.mean(_result):.3f} +/- {np.std(_result):.3f}')
    else: print(f'{i}/{n_tests}')
    learn = ts_learner(dls100, TSTPlus, pretrained=True, weights_path=f'data/MVP/TSTPlus.pth', metrics=mae,res_dropout=0.3, fc_dropout=0.9)
    learn.fine_tune(n_epochs, base_lr=2e-2, freeze_epochs=freeze_epochs)
    _result.append(learn.recorder.values[-1][-1])
learn.plot_metrics()
print(f'\nMAE: {np.mean(_result):.3f} +/- {np.std(_result):.3f} in {n_tests} tests')

In [ ]:
_, test_targets, test_preds = learn.get_preds(dl=test_dl, with_decoded=True, save_preds=None, save_targs=None)
abs(test_preds-test_targets)

# Coding Exercise 4
Repeat for another dataset from monah list or UCR list for RNNs or another model
https://timeseriesai.github.io/tsai/data.external.html

In [ ]:
dsid = 'Cricket' 
X, y, splits = get_UCR_data(dsid, split_data=False)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
get_UCR_multivariate_list()

In [ ]:
get_UCR_univariate_list()